In [493]:
import librosa
import numpy as np
import soundfile as sf
import matplotlib as plt
import torch

In [566]:
# 学習データセットの読み込み
# import pickle
# f = open('soundDataset_20240619.pickle','rb')
# soundDataset_new = pickle.load(f)

# patterns = soundDataset_new.shape[0] #物体の種類
# trials = soundDataset_new.shape[1] #試行数
# len_data = soundDataset_new.shape[2] #1試行の長さ,サンプル数
# num_data = patterns*trials #全試行数

# print(patterns,trials,len_data,num_data)

21 50 66150 1050


In [579]:
# データの読み込み
# path_audio_file = "./data/origin_data/origindata_20240619/01_Y-shirt/01_Y-shirt.WAV"

sr = 44100
test_data = librosa.load(path_audio_file,sr=sr)
print(test_data[0].shape)

#データのトリミング
origin_data = test_data[0]
flag_amp = 0.1 #各試行の合図を検知する基準振幅
trimSkip = int(sr*0.4)
trimTime = int(sr*1.5)#1試行あたりのデータ長
index = 0
while origin_data[index] >= flag_amp:
    trimData = np.array(origin_data[index+trimSkip:index+trimTime+trimSkip])
    index+=1
print(trimData.shape)

#データの保存
output_path = "./demo/trim_datas/sample.wav"
sf.write(output_path,trimData,sr)

trimData = soundDataset_new[11][10]

#メルスペクトログラムに変換
spectrogram = librosa.feature.melspectrogram(y=trimData, sr=sr)# スペクトログラムを計算
spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
print(spectrogram_db.shape)

#tensorに変換
melspec_tensor = torch.tensor(spectrogram_db, dtype=torch.float32)
# test_loader = torch.utils.data.DataLoader(melspec_tensor)
test_loader = melspec_tensor.unsqueeze(0).unsqueeze(0)
print(test_loader.shape)

(9097600,)
(66150,)
(128, 130)
torch.Size([1, 1, 128, 130])


In [580]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#推定モデルの読み込み
objModel_path = "./demo/models/objModel.pth"
load_objModel = torch.load(objModel_path)
objModel = load_objModel.eval()

# 検証データでの評価
with torch.no_grad():
        x = test_loader.to(device)
        # print(x)
        y = objModel(x)
        pred = torch.argmax(y[0], dim=1)
        print(pred)
        confidence = torch.max(y[0])
        print(confidence)

cuda:0
tensor([11], device='cuda:0')
tensor(16.2104, device='cuda:0')
